In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold
import math
import scipy
#note： 
#1. As a result it might not be worthwhile to use features related to game clock/quarter of the game。
#2. There is no relationships between number of rushes before and running yards gained。
#3. rushing success larger depends on defender in box, or defender that are close to offensive lineman and attempt 
#   to counter the blocking.
#4. highly drafted player has the same average rushing yards as the rest
#ARI: Arizona Cardinals
#ATL: Atlanta Falcons
#BAL: Baltimore Ravens
#BUF: Buffalo Bills
#CAR: Carolina Panthers
#CHI: Chicago Bears
#CIN: Cincinnati Bengals
#CLE: Cleveland Browns
#DAL: Dallas Cowboys
#DEN: Denver Broncos
#DET: Detroit Lions
#GB: Green Bay Packers
#HOU: Houston Texans
#IND: Indianapolis Colts
#JAX: Jacksonville Jaguars
#KC: Kansas City Chiefs
#MIA: Miami Dolphins
#MIN: Minnesota Vikings
#NE: New England Patriots
#NO: New Orleans Saints
#NYG: New York Giants
#NYJ: New York Jets
#OAK: Oakland Raiders
#PHI: Philadelphia Eagles
#PIT: Pittsburgh Steelers
#SD: San Diego Chargers
#SEA: Seattle Seahawks
#SF: San Francisco 49ers
#STL: Saint Louis Rams
#TB: Tampa Bay Buccaneers
#TEN Tennessee Titans
#WAS: Washington Redskins

In [211]:
train_df = pd.read_csv(r'C:\Users\38980\OneDrive\Desktop\study\kaggle\NFL\train.csv')

In [212]:
#team abbreviations correct
def data_clean(df):
#correct name   
    df.loc[df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
    df.loc[df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
    df.loc[df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
    df.loc[df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
    df.loc[df['FieldPosition'] == 'ARZ', 'FieldPosition'] = 'ARI'
    df.loc[df['FieldPosition'] == 'BLT', 'FieldPosition'] = 'BAL'
    df.loc[df['FieldPosition'] == 'CLV', 'FieldPosition'] = 'CLE'
    df.loc[df['FieldPosition'] == 'HST', 'FieldPosition'] = 'HOU'

# offense time and defence time
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    
#time
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df['GameClock'] = df['GameClock'].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
#player height
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

#weather
    df['Cleaned_GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
#diff Score    
    df["DiffScoreBeforePlay_ob"] = (df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"])
    df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]] = - df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]]
#Turf
    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
    df['Cleaned_Turf'] = df['Turf'].map(Turf)

#left to right
    df['New_X'] = df['X']
    df.loc[df['PlayDirection'] == 'left','New_X'] = 120 - df.loc[df['PlayDirection'] == 'left','X']
    df['New_Y'] = df['Y']
    df.loc[df['PlayDirection'] == 'left','New_Y'] = 160/3 - df.loc[df['PlayDirection'] == 'left','Y']
    df['Orientation_std'] = df['Orientation']
    df.loc[df['Season'] == 2017, 'Orientation_std'] = df.loc[df['Season'] == 2017, 'Orientation_std'] + 90
    df.loc[df['PlayDirection'] == 'left', 'Orientation_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation_std'], 360)
    df['Dir_std'] = df['Dir']
    df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir_std'], 360)
    df['YardLine_std'] = 100 - df['YardLine']
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine_std'
         ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine']
    df["Orientation_sin"] = df["Orientation_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Orientation_cos"] = df["Orientation_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    df["Dir_sin"] = df["Dir_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Dir_cos"] = df["Dir_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

#distance
    #distance to yardline
    df['Dis_YardLine'] = df['New_X'] - df['YardLine_std'] - 10
    #distance to rusher
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    Rusher =df.loc[df['IsRusher'],['PlayId','X','Y','Dir_std']].rename(columns={"X":"Rusher_X","Y":"Rusher_Y",'PossessionTeam':'Offense_Team','Dir_std':'Rusher_Dir_std'})
    df = df.merge(Rusher,how = 'left',on = 'PlayId')
    df['Distance_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_Diff'] = abs(df['Degree_to_Rusher'] - df['Rusher_Dir_std'])
    #back_direction
    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0
    df['back_oriented_down_field'] = df['Orientation_std'].apply(lambda x: back_direction(x))

# Stadium clean
    map_stad = {'Broncos Stadium at Mile High': 'Broncos Stadium At Mile High', 'CenturyField': 'CenturyLink Field', 'CenturyLink': 'CenturyLink Field', 'Everbank Field': 'EverBank Field', 'FirstEnergy': 'First Energy Stadium', 'FirstEnergy Stadium': 'First Energy Stadium', 'FirstEnergyStadium': 'First Energy Stadium', 'Lambeau field': 'Lambeau Field', 'Los Angeles Memorial Coliesum': 'Los Angeles Memorial Coliseum', 'M & T Bank Stadium': 'M&T Bank Stadium', 'M&T Stadium': 'M&T Bank Stadium', 'Mercedes-Benz Dome': 'Mercedes-Benz Superdome', 'MetLife': 'MetLife Stadium', 'Metlife Stadium': 'MetLife Stadium', 'NRG': 'NRG Stadium', 'Oakland Alameda-County Coliseum': 'Oakland-Alameda County Coliseum', 'Paul Brown Stdium': 'Paul Brown Stadium', 'Twickenham': 'Twickenham Stadium'}
    
    for stad in df['Stadium'].unique():
        if stad in map_stad.keys():
            pass
        else:
            map_stad[stad]=stad

    df['Stadium'] = df['Stadium'].map(map_stad)

# wind 
    def give_me_WindSpeed(x):
            x = str(x)
            x = x.replace('mph', '').strip()
            if '-' in x:
                x = (int(x.split('-')[0]) + int(x.split('-')[1])) / 2
            elif 'gusts up to' in x:
                x = (int(x.split()[0]) + int(x.split()[-1])) / 2
            elif 'clam' in x:
                x = 0
            try:
                return float(x)
            except:
                return -99
    df['Cleaned_WindSpeed'] = df['WindSpeed'].apply(give_me_WindSpeed)

# speed
    df['Horizontal Speed'] = df['S']*df["Dir_sin"]
    df['Vertical Speed'] = df['S']*df["Dir_cos"]
    
    return df

In [213]:
train = data_clean(train_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in remainder
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in remainder
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: divide by zero encountered in double_scalars


In [214]:
def rusher_game(df):
    df1 = df.loc[df['IsRusher']]
    df2 = df.loc[df['IsOnOffense'] & (~df['IsRusher'])]
    df3 = df.loc[~df['IsOnOffense']]
    
# min_time_to_tacke
    df3['Min_Time_Tacke'] = df3['distance_to_rusher']/df3['S']

# defence_X_Y_spread
    Defence_X_Y_std = df3[["PlayId",'New_X','New_Y']].groupby("PlayId").std().rename(columns={'New_X':'Defense_X_std','New_Y':'Defense_Y_std'}) \
    .reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed2_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(1).index, df3.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed2_std'}).reset_index()

    df3 = df3.sort_values(['PlayId','New_Y'])
    Defense_Y_Removed2_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(1).index, df3.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed2_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed4_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed4_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_Y'])
    Defense_Y_Removed4_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Defence_X_Y_std, how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed4_std,how = 'left',  on ='PlayId')

# offense_X_Y_spread
    df2 = df2.sort_values(['PlayId','New_X'])
    Offense_X_Removed2_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed2_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed2_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed2_std'}).reset_index()
    
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_X_Removed4_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed4_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed4_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Offense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed4_std,how = 'left',  on ='PlayId')
    
# nearest offenders to defender
    dis_to_closest_offender = pd.DataFrame()
    for playid in df3['PlayId'].unique():
        offense = df2.loc[df2['PlayId'] == playid]
        defence = df3.loc[df3['PlayId'] == playid]
        ary = scipy.spatial.distance.cdist(defence[['New_X','New_Y']], offense[['New_X','New_Y']], metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, defence[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_offender = dis_to_closest_offender.append(ary)
    df3 = df3.merge(dis_to_closest_offender,how = 'left',on = ['PlayId','NflId'])
    
    return df1, df2, df3 

In [196]:
rusher, offender, defender = rusher_game(train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [133]:
df1 = pd.DataFrame({'lkey': ['foo', 'foo', 'baz', 'baz'],'value': [1,2,1,2],'abc':[1,2,3,4]})
df2 = pd.DataFrame({'lkey': ['foo', 'foo', 'baz', 'baz'],'value': [1,2,1,2],'abc':[1,2,3,4]})


In [134]:
df1.merge(df2, on=['lkey','value'])

,lkey,value,abc_x,abc_y
0,foo,1,1,1
1,foo,2,2,2
2,baz,1,3,3
3,baz,2,4,4


In [ ]:
df2 = df2.sort_values(['PlayId','New_X'])
df2["PlayId",'New_X'].drop(df2.groupby('PlayId').tail(1).index + df2.groupby('PlayId').head(1).index) \
.groupby('PlayId').std

In [147]:
sample = train.loc[train['PlayId'] == 20181230153888]
df1 = sample.loc[train['IsRusher']]
df2 = sample.loc[train['IsOnOffense']]
df3 = sample.loc[~train['IsOnOffense']]
ary = scipy.spatial.distance.cdist(df2[['New_X','New_Y']], df3[['New_X','New_Y']], metric='euclidean')
ary.sort(axis=1)
ary = pd.DataFrame(data = ary)
ary


,0,1,2,3,4,5,6,7,8,9,10
0,1.211156,1.519539,2.033937,2.080817,4.145841,5.140623,5.958297,7.635188,8.274503,12.617646,21.483298
1,1.205031,2.075307,2.608639,3.278079,5.810955,6.781866,6.933542,7.752193,10.077549,13.634082,23.274074
2,1.030049,1.275813,1.816398,2.334524,3.456487,4.670685,5.480146,7.630531,7.751264,11.967159,20.990836
3,1.403460,1.887671,2.684772,2.869373,3.127507,3.653081,4.174326,5.866524,8.947067,11.120112,19.116700
4,3.215727,4.271452,4.852793,5.318064,5.793142,6.089499,6.154649,7.030797,11.808819,15.152944,19.291822
5,1.221556,1.650000,2.601096,4.561107,4.943127,7.023874,7.536717,8.415759,12.769123,13.329666,15.162566
6,0.689493,1.020784,1.504161,2.828516,4.871150,6.340954,6.366223,7.150007,9.409384,12.527526,22.655176
7,3.430233,4.263109,4.810094,5.560576,5.912165,6.349378,6.392535,6.944012,12.063913,15.267144,19.105481
8,0.672086,0.703278,2.955757,3.605010,4.072407,6.075689,6.562202,7.496092,12.132720,12.338590,15.939793
9,0.837377,1.593142,2.971363,3.032755,3.968501,5.013232,5.543014,6.410686,11.374502,12.082525,17.037444


In [48]:
train.loc[train['PlayId'] == 20181230153888,['PossessionTeam']]

,PossessionTeam
509630,ARI
509631,ARI
509632,ARI
509633,ARI
509634,ARI
509635,ARI
509636,ARI
509637,ARI
509638,ARI
509639,ARI


In [192]:
dis_to_closest_offender = pd.DataFrame()
dis_to_closest_offender = dis_to_closest_offender.append(ary)
dis_to_closest_offender

,0,1,2,3,4,5,6,7,8,9,10
0,1.211156,1.519539,2.033937,2.080817,4.145841,5.140623,5.958297,7.635188,8.274503,12.617646,21.483298
1,1.205031,2.075307,2.608639,3.278079,5.810955,6.781866,6.933542,7.752193,10.077549,13.634082,23.274074
2,1.030049,1.275813,1.816398,2.334524,3.456487,4.670685,5.480146,7.630531,7.751264,11.967159,20.990836
3,1.403460,1.887671,2.684772,2.869373,3.127507,3.653081,4.174326,5.866524,8.947067,11.120112,19.116700
4,3.215727,4.271452,4.852793,5.318064,5.793142,6.089499,6.154649,7.030797,11.808819,15.152944,19.291822
5,1.221556,1.650000,2.601096,4.561107,4.943127,7.023874,7.536717,8.415759,12.769123,13.329666,15.162566
6,0.689493,1.020784,1.504161,2.828516,4.871150,6.340954,6.366223,7.150007,9.409384,12.527526,22.655176
7,3.430233,4.263109,4.810094,5.560576,5.912165,6.349378,6.392535,6.944012,12.063913,15.267144,19.105481
8,0.672086,0.703278,2.955757,3.605010,4.072407,6.075689,6.562202,7.496092,12.132720,12.338590,15.939793
9,0.837377,1.593142,2.971363,3.032755,3.968501,5.013232,5.543014,6.410686,11.374502,12.082525,17.037444


In [73]:
df3

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,TeamOnOffense,IsOnOffense,PlayerAge,TimeDelta,Cleaned_GameWeather,DiffScoreBeforePlay_ob,Cleaned_Turf,New_X,New_Y,Orientation_std,Dir_std,YardLine_std,Orientation_sin,Orientation_cos,Dir_sin,Dir_cos,Dis_YardLine,IsRusher,Rusher_X,Rusher_Y,distance_to_rusher,back_oriented_down_field,Cleaned_WindSpeed,Horizontal Speed,Vertical Speed
509641,2018123015,20181230153888,home,44.71,31.09,1.26,2.37,0.13,91.24,151.48,2495252,K.J. Wright,50,2018,36,4,3.733333,ARI,3,1,SEA,24,21,2553435,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",left,2018-12-31 00:13:16,2018-12-31 00:13:15,1,76,246,1989-07-23,Mississippi State,OLB,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE,away,False,29.440135,1.0,Cloudy,3,Artificial,75.29,22.243333,271.24,331.48,64,-0.999766,0.021640,-0.477465,0.878650,1.29,False,49.8,27.63,26.019479,1,5.0,-0.601607,1.107100
509642,2018123015,20181230153888,home,44.02,27.24,2.81,3.23,0.26,76.70,95.46,2532966,Bobby Wagner,54,2018,36,4,3.733333,ARI,3,1,SEA,24,21,2553435,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",left,2018-12-31 00:13:16,2018-12-31 00:13:15,1,72,242,1990-06-27,Utah State,MLB,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE,away,False,28.512003,1.0,Cloudy,3,Artificial,75.98,26.093333,256.70,275.46,64,-0.973179,-0.230050,-0.995463,0.095151,1.98,False,49.8,27.63,27.804268,1,5.0,-2.797251,0.267374
509643,2018123015,20181230153888,home,34.69,26.49,1.34,3.69,0.14,65.64,29.05,2539243,Bradley McDougald,30,2018,36,4,3.733333,ARI,3,1,SEA,24,21,2553435,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",left,2018-12-31 00:13:16,2018-12-31 00:13:15,1,73,215,1990-11-15,Kansas,SS,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE,away,False,28.125966,1.0,Cloudy,3,Artificial,85.31,26.843333,245.64,209.05,64,-0.910972,-0.412469,-0.485573,-0.874196,11.31,False,49.8,27.63,23.637870,1,5.0,-0.650667,-1.171423
509644,2018123015,20181230153888,home,45.25,22.27,1.86,3.63,0.17,70.40,341.25,2540140,Barkevious Mingo,51,2018,36,4,3.733333,ARI,3,1,SEA,24,21,2553435,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",left,2018-12-31 00:13:16,2018-12-31 00:13:15,1,77,235,1990-10-04,Louisiana State,OLB,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE,away,False,28.240956,1.0,Cloudy,3,Artificial,74.75,31.063333,250.40,161.25,64,-0.942057,-0.335452,0.321439,-0.946930,0.75,False,49.8,27.63,31.931008,1,5.0,0.597877,-1.761290
509645,2018123015,20181230153888,home,46.22,25.30,0.50,3.01,0.07,52.25,122.70,2543706,Shamar Stephen,98,2018,36,4,3.733333,ARI,3,1,SEA,24,21,2553435,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",left,2018-12-31 00:13:16,2018-12-31 00:13:15,1,77,310,1991-02-25,Connecticut,NT,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE,away,False,27.846706,1.0,Cloudy,3,Artificial,73.78,28.033333,232.25,302.70,64,-0.790690,-0.612217,-0.841511,0.540240,-0.22,False,49.8,27.63,30.482049,1,5.0,-0.420755,0.270120
509646,2018123015,20181230153888,home,45.53,30.80,0.53,0.34,0.06,111.64,121.54,2552629,Frank Clark,55,2018,36,4,3.733333,ARI,3,1,SEA,24,21,2553435,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",left,2018-12-31 00:13:16,2018-12-31 00:13:15,1,75,260,1993-06-14,Michigan,DE,SEA,ARI,17,CenturyLink Field,"Seattle, WA",Outdoor,FieldTurf,Cloudy,45.0,76.0,5,SE,away,False,25.546911,1.0,Cloudy,3,Artificial,74.47,22.533333,291.64,301.54,64,-0.929519,0

In [11]:
def get_dx_dy(angle, dist):
    radian_angle = (90 - angle)* math.pi/180.0
    dx = dist * math.cos(radian_angle)
    dy = dist * math.sin(radian_angle)
    return dx, dy